In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import os
import librosa as lb
import glob
from pprint import pprint as pp
import scipy
import matplotlib.pyplot as plt
from  more_itertools import unique_everseen
from collections import OrderedDict
from pydub import utils
import math
import statistics
import itertools
import csv

In [2]:
# Path for audio data
data ={}
name_list =[]
path = r'C:\\Users\\timur\\OneDrive\\Documents\\R Studio Tasks\\ДИПЛОМ\\Audio New\\'

files = [f for f in glob.glob(path + "*.mp3", recursive=True)]

# Since it is not fast and sometimes it crushes - I divided the extraction by several file packs 
files0 = files[:4250:]
files1 = files[4250:8500:]
files2 = files[8500:13059:]
files3 = files[8500+2800:13059:]

In [3]:
data ={}
name_list =[]

for file in files0:
    # Show the name of the audio file
    print(file + "                                                                                        ", end = "\r")
    
    # Create name for ds appearance
    name = file.split('\\')[-1].split('.mp3')[0]
    name_list.append(name)

    # Audio -> Python numbers
    dur = utils.mediainfo(file)["duration"]
    y, sr = lb.load(file, sr=44100, duration = math.floor(float(dur)))
    data[name] = {'y' : y, 'sr': sr}


c:\Users\timur\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


In [4]:
def get_audio_features(song, sr, hop_length, n_fft):

    y_harmonic, y_percussive = lb.effects.hpss(song)  # split song into harmonic and percussive parts
    stft_harmonic = lb.core.stft(y_harmonic, n_fft=n_fft, hop_length=hop_length)  # Compute power spectrogram.
    stft_percussive = lb.core.stft(y_percussive, n_fft=n_fft, hop_length=hop_length)  # Compute power spectrogram.

    rmsH = np.sqrt(np.median(np.abs(lb.feature.rms(S=stft_harmonic)) ** 2, axis=0, keepdims=True))
    rmsH_median = np.median(rmsH)
    rmsH_std = np.std(rmsH)

    rmsP = np.sqrt(np.median(np.abs(lb.feature.rms(S=stft_percussive)) ** 2, axis=0, keepdims=True))
    rmsP_median = np.median(rmsP)
    rmsP_std = np.std(rmsP)

    centroid = lb.feature.spectral_centroid(song, sr, n_fft=n_fft,
                                            hop_length=hop_length)  # Compute the spectral centroid.
    centroid_median = np.median(centroid)
    centroid_std = np.std(centroid)

    bw = lb.feature.spectral_bandwidth(song, sr, n_fft=n_fft,
                                       hop_length=hop_length)  # Compute p’th-order spectral bandwidth:
    bw_median = np.median(bw)
    bw_std = np.std(bw)

    contrast = lb.feature.spectral_contrast(song, sr, n_fft=n_fft,
                                            hop_length=hop_length)  # Compute spectral contrast
    contrast_median = np.median(contrast)
    contrast_std = np.std(contrast)

    polyfeat = lb.feature.poly_features(y_harmonic, sr, n_fft=n_fft,
                                        hop_length=hop_length)  # Get coefficients of fitting an nth-order polynomial to the columns of a spectrogram.
    polyfeat_median = np.median(polyfeat[0])
    polyfeat_std = np.std(polyfeat[0])

    tonnetz = lb.feature.tonnetz(lb.effects.harmonic(y_harmonic),
                                 sr)  # Computes the tonal centroid features (tonnetz), following the method of
    tonnetz_median = np.median(tonnetz)
    tonnetz_std = np.std(tonnetz)

    zcr = lb.feature.zero_crossing_rate(song, sr, hop_length=hop_length)  # zero crossing rate
    zcr_median = np.median(zcr)

    features_dict = OrderedDict({'rmseP_median': rmsP_median, 'rmseP_std': rmsP_std, 
                                 'rmseH_median': rmsH_median, 'rmseH_std': rmsH_std,
                                 'centroid_median': centroid_median, 'centroid_std': centroid_std, 
                                 'bw_std': bw_std,
                                 'contrast_median': contrast_median, 'contrast_std': contrast_std, 
                                 'polyfeat_median': polyfeat_median, 'polyfeat_std': polyfeat_std, 
                                 'tonnetz_median': tonnetz_median, 'tonnetz_std': tonnetz_std,
                                 'zcr_median': zcr_median})

    return features_dict

In [5]:
import warnings
from tqdm import tqdm

# final dictionary
od_combined = OrderedDict()

# Function loop
for i in tqdm(range(len(name_list))):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        
        # The actual function
        od = get_audio_features(data[name_list[i]]['y'], data[name_list[i]]['sr'], hop_length=512, n_fft=2048)
    
    # To the dictionary
    od['song_name'] = name_list[i]
    for key, value in od.items():
        if key not in od_combined:
            od_combined[key] = []
        od_combined[key].append(value)

    # Download result every 100 tracks
    if i % 100 == 0:
        cffa441 = pd.DataFrame(od_combined, columns=od_combined.keys())
        cffa441.to_csv('new_combine_features_full_audio_final_3.csv')

print('finish')

100%|██████████| 1759/1759 [3:02:29<00:00,  6.22s/it]  

finish
